In [5]:
!git clone https://github.com/Pritam3355/Datasets.git

Cloning into 'Datasets'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 16 (delta 5), reused 6 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (16/16), 200.04 KiB | 3.39 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [6]:
!pip install -q sdv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.0/154.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 5.5 MB/s eta 0:00:00


In [7]:
import pandas as pd
import numpy as np

fpath = "/content/Datasets/Fundamental_Data.csv"
df = pd.read_csv(fpath)


In [8]:
tickers = df.stock_symbol.unique().tolist()
tickers = tickers[:10]
df_2016_2020 = df[df.stock_symbol.isin(tickers) & df.year.isin([2016,2017,2018,2019,2020])]
df_2020_2024 = df[df.stock_symbol.isin(tickers) & df.year.isin([2020,2021,2022,2023,2024])]
df_2016_2020.shape, df_2020_2024.shape

((1300, 6), (3536, 6))

In [9]:
category_1 = df_2016_2020.category.unique().tolist()
category_2 = df_2020_2024.category.unique().tolist()
set_diff = list(set(category_2) - set(category_1))
set_diff

['Gross NPA %',
 'Other Income_quarters',
 'promoters_quarterly',
 'Sales_quarters',
 'Expenses_quarters',
 'Net Profit_quarters',
 'public_quarterly',
 'No. of Shareholders',
 'foreign_institutions_quarterly',
 'domestic_institutions_quarterly',
 'Net NPA %',
 'government_quarterly',
 'others_quarterly']

In [10]:
mask = df_2020_2024["category"].isin(set_diff)
df_2020_2024 = df_2020_2024[~mask]
df_2020_2024 = df_2020_2024[df_2020_2024.month==3]
category_2 = df_2020_2024.category.unique().tolist()
set_diff = list(set(category_2) - set(category_1))
set_diff

[]

In [11]:
df_2016_2020.shape ,df_2020_2024.shape

((1300, 6), (1516, 6))

In [33]:
import sdv

from sdv.metadata import Metadata


metadata = Metadata.detect_from_dataframe(data=df_2016_2020)

metadata.update_column(
    column_name='value_type',
    sdtype='numerical'
)

metadata.validate()
print(metadata,type(metadata))

{
    "tables": {
        "table": {
            "columns": {
                "category": {
                    "sdtype": "categorical"
                },
                "report_type": {
                    "sdtype": "categorical"
                },
                "stock_symbol": {
                    "sdtype": "categorical"
                },
                "year": {
                    "sdtype": "categorical"
                },
                "month": {
                    "sdtype": "categorical"
                },
                "value_type": {
                    "sdtype": "numerical"
                }
            }
        }
    },
    "relationships": [],
    "METADATA_SPEC_VERSION": "V1"
} <class 'sdv.metadata.metadata.Metadata'>


In [34]:
from sdv.single_table import CTGANSynthesizer

synthesizer = CTGANSynthesizer(
    metadata, # required
    enforce_rounding=False,
    epochs=200,
    cuda=False,
    verbose=True
)


/usr/local/lib/python3.10/dist-packages/sdv/single_table/base.py:104: UserWarning:

We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.



In [35]:
synthesizer.fit(df_2016_2020)
synthetic_data = synthesizer.sample(num_rows=1667)

Gen. (-0.12) | Discrim. (-0.07): 100%|██████████| 200/200 [00:35<00:00,  5.64it/s]


In [15]:
!pip install -q rdt[copulas]

In [36]:
# synthesizer.auto_assign_transformers(pandas_tmp_df_2016_2020)

from rdt.transformers.numerical import ClusterBasedNormalizer
from rdt.transformers.numerical import GaussianNormalizer



synthesizer.update_transformers(column_name_to_transformer={
    'value_type': GaussianNormalizer()
})
processed_data = synthesizer.preprocess(df_2016_2020)
synthesizer.fit_processed_data(processed_data)

/usr/local/lib/python3.10/dist-packages/rdt/transformers/numerical.py:355: FutureWarning:

Future versions of RDT will not support 'truncated_gaussian' as an option. Please use 'truncnorm' instead.

/usr/local/lib/python3.10/dist-packages/sdv/single_table/base.py:293: UserWarning:

For this change to take effect, please refit the synthesizer using `fit`.

/usr/local/lib/python3.10/dist-packages/sdv/single_table/base.py:452: UserWarning:

This model has already been fitted. To use the new preprocessed data, please refit the model using 'fit' or 'fit_processed_data'.

Gen. (-0.27) | Discrim. (0.05): 100%|██████████| 200/200 [00:37<00:00,  5.32it/s]


In [37]:
synthesizer.fit(df_2016_2020)
synthetic_data_2= synthesizer.sample(num_rows=1516)
synthetic_data_2.shape

Gen. (0.05) | Discrim. (0.00): 100%|██████████| 200/200 [00:37<00:00,  5.37it/s]


(1516, 6)

In [38]:
synthesizer.get_loss_values()
fig = synthesizer.get_loss_values_plot()
fig.show()

In [39]:
columns_info = metadata.to_dict()['tables']['table']['columns']

modified_metadata = {
    'columns': columns_info,
    'relationships': metadata.to_dict()['relationships'],
    'METADATA_SPEC_VERSION': metadata.to_dict()['METADATA_SPEC_VERSION']
}

In [40]:
from sdmetrics.reports.single_table import DiagnosticReport

report = DiagnosticReport()
report.generate(df_2020_2024,synthetic_data_2, modified_metadata)
report.get_details(property_name='Data Validity')


Generating report ...

(1/2) Evaluating Data Validity: |██████████| 6/6 [00:00<00:00, 301.17it/s]|
Data Validity Score: 86.18%

(2/2) Evaluating Data Structure: |██████████| 1/1 [00:00<00:00, 123.41it/s]|
Data Structure Score: 100.0%

Overall Score (Average): 93.09%



,Column,Metric,Score
0,category,CategoryAdherence,1.000000
1,report_type,CategoryAdherence,1.000000
2,stock_symbol,CategoryAdherence,1.000000
3,year,CategoryAdherence,0.170844
4,month,CategoryAdherence,1.000000
5,value_type,BoundaryAdherence,1.000000


In [41]:
fig = report.get_visualization(property_name='Data Validity')
fig.show()

In [42]:
from sdmetrics.single_column import StatisticSimilarity

StatisticSimilarity.compute(
    real_data=df_2020_2024['value_type'],
    synthetic_data=synthetic_data_2['value_type'],
    statistic='std'
)

0.9840376436947553

In [43]:
from sdv.evaluation.single_table import evaluate_quality

quality_report = evaluate_quality(
    df_2020_2024,
    synthetic_data_2,
    metadata
)

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 6/6 [00:00<00:00, 234.42it/s]|
Column Shapes Score: 75.12%

(2/2) Evaluating Column Pair Trends: |██████████| 15/15 [00:00<00:00, 70.19it/s]|
Column Pair Trends Score: 57.98%

Overall Score (Average): 66.55%



In [44]:
quality_report.get_details('Column Shapes')

,Column,Metric,Score
0,category,TVComplement,0.872032
1,report_type,TVComplement,0.804090
2,stock_symbol,TVComplement,0.965699
3,year,TVComplement,0.170844
4,month,TVComplement,1.000000
5,value_type,KSComplement,0.694591


In [45]:
from sdv.evaluation.single_table import get_column_plot

fig = get_column_plot(
    real_data=df_2020_2024,
    synthetic_data=synthetic_data_2,
    column_name='value_type',
    metadata=metadata
)

fig.show()

In [46]:
from sdv.evaluation.single_table import get_column_pair_plot

fig = get_column_pair_plot(
    real_data=df_2020_2024,
    synthetic_data=synthetic_data_2,
    column_names=['value_type', 'category'],
    metadata=metadata
)

fig.show()

In [47]:
fig = get_column_pair_plot(
    real_data=df_2020_2024,
    synthetic_data=synthetic_data_2,
    column_names=['stock_symbol', 'category'],
    metadata=metadata
)

fig.show()

In [48]:
synthetic_data_2.groupby('stock_symbol').count()

,category,report_type,year,month,value_type
stock_symbol,,,,,
ADANIENT,160,160,160,160,160
ADANIPORTS,149,149,149,149,149
APOLLOHOSP,144,144,144,144,144
ASIANPAINT,166,166,166,166,166
AXISBANK,142,142,142,142,142
BAJAJ-AUTO,141,141,141,141,141
BAJAJFINSV,157,157,157,157,157
BAJFINANCE,150,150,150,150,150
BHARTIARTL,134,134,134,134,134


In [49]:
df_2020_2024.groupby('stock_symbol').count()

,category,report_type,year,month,value_type
stock_symbol,,,,,
ADANIENT,151,151,151,151,151
ADANIPORTS,151,151,151,151,151
APOLLOHOSP,151,151,151,151,151
ASIANPAINT,151,151,151,151,151
AXISBANK,154,154,154,154,154
BAJAJ-AUTO,151,151,151,151,151
BAJAJFINSV,151,151,151,151,151
BAJFINANCE,154,154,154,154,154
BHARTIARTL,151,151,151,151,151
